# Construction of a prediction model for Covid19
## Part 2 of the notebook (for part one look at the fork of covidclinicaldata

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

from colorama import Fore #To pain the terminal with different colors
import warnings
warnings.filterwarnings('ignore') #To disable warnings
from IPython.display import display #To print a dataframe like the cell does

In [2]:
all_data = pd.read_csv("covid_clinical_data.csv", index_col=0)
all_data

,covid19_test_results,age,high_risk_exposure_occupation,high_risk_interactions,diabetes,chd,htn,cancer,asthma,autoimmune_dis,...,sob,sob_severity,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat
0,Negative,4,True,NaN,False,False,False,False,False,False,...,False,0.0,False,False,False,False,False,False,False,False
1,Negative,2,False,NaN,False,False,False,False,False,False,...,False,0.0,False,False,False,False,False,False,False,False
2,Negative,1,NaN,NaN,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Negative,3,True,True,False,False,False,False,False,False,...,True,2.0,False,True,False,False,False,False,False,True
4,Negative,1,False,NaN,False,False,False,False,False,False,...,False,0.0,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93989,Negative,3,False,True,False,False,False,False,False,False,...,False,0.0,False,False,False,False,False,False,False,False
93990,Negative,3,False,True,False,False,False,False,False,False,...,False,0.0,False,False,False,False,False,True,False,True
93991,Negative,3,False,False,False,False,False,False,False,False,...,False,0.0,False,False,False,False,False,False,False,False
93993,Negative,3,False,False,False,False,False,False,False,False,...,False,0.0,False,False,False,False,False,False,False,False


In [3]:
X = all_data.iloc[:, 1:]
y = all_data["covid19_test_results"]

In [95]:
def create_different_datasets(data_x):
    e_factors = data_x.iloc[:,:3]
    comorbidities_data = data_x.iloc[:, 3:10] #The columns that contain the comorbidities
    vitals_data = data_x.iloc[:, 10:16] #The columns that contain the vitals
    assesed_symptoms_data = data_x.iloc[:, 1:21] #The columns that contain the assesed symptoms
    reported_symptoms_data = data_x.iloc[:,21:] #The columns that have the patient reported symptoms
    
    return e_factors, comorbidities_data, vitals_data, assesed_symptoms_data, reported_symptoms_data

In [96]:
e_factors, comorb, vitals, a_symptoms, r_symptoms = create_different_datasets(X)

<u>high_risk_exposure_occupation:</u> The most obvious thing to do is to just fill the 169 missing values with the most frequent value. However, it occurred to me that since we ultimately are willing to have (maybe even encourage) false positives, it might be better to just put the ones that tested positive as True and the rest as False. This would require that I split the dataset into training, validation and testing set before I do any imputation to avoid any target leakage (the effect when knowing the result before hand affects how are we imputing the variables). The step to be taken (if I were to follow that route) would be then to impute the training data that tested positive as True, the rest as False, and any incoming unknown data (validation and test data) as True if we dont know the value of the feature. I might need to run an experiment when I do both.

<u>high_risk_interaction:</u> This is easier than the previous one. We will just assign True to anything that has a 'high_risk_expossure_occupation' as True. The reasoning behind is is that if we don't know if the patient has had a high risk interaction, makes sense to say they did if their occupation is of high risk exposure.

In [6]:
def high_risk_exposure_imputation(data_x, data_y, target_imputation=False):
    if target_imputation:
        mode = data_x[data_y == "Positive"].high_risk_exposure_occupation.mode()[0]
    else:
        mode = data_x.high_risk_exposure_occupation.mode()[0]
    data_x.loc[data_x["high_risk_exposure_occupation"].isna(), "high_risk_exposure_occupation"] = mode
    
    data_x.loc[data_x["high_risk_interactions"].isna(), "high_risk_interactions"] = data_x["high_risk_exposure_occupation"]

In [7]:
def reduce_training_set(data_x, data_y):
    selected_negatives = data_x[data_y[data_x.index] == "Negative"].sample((data_y[data_x.index] == "Positive").sum())
    all_positives = data_x[data_y == "Positive"]
    #reduced_train_comorb = pd.merge(all_positives, selected_negatives) #Doesn't work well for reasons...
    reduced_data = pd.concat([all_positives, selected_negatives])

    return reduced_data

def get_metrics(*values):
    accuracy = accuracy_score(*values)
    recall = recall_score(*values, pos_label="Positive")
    precision = precision_score(*values, pos_label="Positive")
    f1 = f1_score(*values, pos_label="Positive")
    return accuracy, recall, precision, f1


In [8]:
class XGBAdapter:
    
    def __init__(self, **params):
        self.model = XGBClassifier(**params, verbosity=0)
        
    def fit(self, data_x, data_y):
        self.model.fit(self.xgb_adapter(data_x), data_y)
        
    def predict(self, data_x):
        return self.model.predict(self.xgb_adapter(data_x))
        
    #This function is because xgb complains about 'object' type columns so I convert them all to boolean
    def xgb_adapter(self, data_x):
        result = data_x.copy()
        for column in result:
            result.loc[:,column] = result[column].astype("bool")
        return result

In [85]:
def cross_validation_normal(data_x, data_y, model, folds, hr_imputation=False):
    results = np.zeros((folds,2,4)) #Shape of folds, training and validation, and number of metrics
    
    fold_size = data_x.shape[0] // folds
    reminder = data_x.shape[0] % folds    
    start = 0    
    for i in range(folds):
        end = start + fold_size + (1 if reminder > 0 else 0)
        reminder-=1
        train_x = pd.concat([data_x.iloc[:start], data_x.iloc[end:]], axis=0)
        train_y = data_y.iloc[:start].append(data_y.iloc[end:])
        valid_x = data_x.iloc[start:end]
        valid_y = data_y.iloc[start:end]
        
        if hr_imputation:
            high_risk_exposure_imputation(train_x, train_y, target_imputation = True)
            high_risk_exposure_imputation(valid_x, valid_y)
            
        model.fit(train_x, train_y)
        train_v = (train_y, model.predict(train_x))
        valid_v = (valid_y, model.predict(valid_x))
        results[i,0] += get_metrics(*train_v)
        results[i,1] += get_metrics(*valid_v)
        start = end
        
        progress_bar = "╠" + str("■" * i) + str(" " * (folds -(i+1))) + "╣"
        print(progress_bar, end="\r")
    print("")
    return results.mean(axis=0)

def cross_validation_class_ratios(data_x, data_y, model, folds, hr_imputation=False, ratio_negatives_to_positives=1):
    
    positive_index = data_y[data_y == "Positive"].index
    size_of_negatives = positive_index.shape[0] * ratio_negatives_to_positives
    
    fold_size = positive_index.shape[0] // folds
    results = np.zeros((folds,2,4)) #Shape of folds, training and validation, and number of metrics   
        
    reminder = positive_index.shape[0] % folds
    
    start = 0    
    for i in range(folds):
        end = start + fold_size + (1 if reminder > 0 else 0)
        reminder-=1        
        training_positives = np.setdiff1d(positive_index, positive_index[start:end]) #The index of positives to be used as training set
        
        # Sample the size of negatives- the size of the fold negatives for training and concatenate with the corresponding indexes for the fold
        train_x = pd.concat([data_x.sample(size_of_negatives - fold_size), data_x.loc[training_positives]], axis=0) # 'loc' instead of 'iloc' because the indexes are treated as labels, not numbers
        train_y = data_y[train_x.index]
        valid_x = pd.concat([data_x.sample(fold_size * ratio_negatives_to_positives), data_x.loc[positive_index[start:end]]], axis=0) # 'loc' instead of 'iloc' because the indexes are treated as labels, not numbers
        valid_y = data_y[valid_x.index]
        
        if hr_imputation:
            high_risk_exposure_imputation(train_x, train_y, target_imputation = True)
            high_risk_exposure_imputation(valid_x, valid_y)
            
        model.fit(train_x, train_y)
        train_v = (train_y, model.predict(train_x))
        valid_v = (valid_y, model.predict(valid_x))
        results[i,0] += get_metrics(*train_v)
        results[i,1] += get_metrics(*valid_v)
        start = end
        
        progress_bar = "╠" + str("■" * i) + str(" " * (folds -(i+1))) + "╣"
        print(progress_bar, end="\r")
    print("")
    return results.mean(axis=0)
    
        
def run_models_cross_val(data_x, data_y, models, folds=10, hr_imputation=False, ratio_negatives_to_positives=-1):
    for model in models:
        print(Fore.RED, model, Fore.BLACK, sep="")
        if(ratio_negatives_to_positives >= 1):            
            results = cross_validation_class_ratios(data_x, data_y, models[model], folds, hr_imputation, ratio_negatives_to_positives)
        else:
            results = cross_validation_normal(data_x, data_y, models[model], folds, hr_imputation)    
            
        display(pd.DataFrame(results, index=["Training", "Validation"], columns=["Accuracy", "Recall", "Precision", "F1"]))
        print("")

In [110]:
models ={
    "MultinomialNB" : MultinomialNB(alpha=0, class_prior=(.5, .5)),
    "Random Forest (100 estimators)" : RandomForestClassifier(min_samples_leaf=30),
    "Random Forest (500 estimators)" : RandomForestClassifier(n_estimators = 500),
    "XGBoost Classifier" : XGBAdapter(n_estimators=500),
    "SVM" : SVC()    
}

In [102]:
run_models_cross_val(comorb, y[comorb.index], models)

MultinomialNB
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.904953,0.083474,0.026178,0.039854
Validation,0.905138,0.082413,0.024478,0.037117



Random Forest (100 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976361,0.0,0.0,0.0
Validation,0.976362,0.0,0.0,0.0



Random Forest (500 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976361,0.0,0.0,0.0
Validation,0.976362,0.0,0.0,0.0



XGBoost Classifier
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976361,0.0,0.0,0.0
Validation,0.976362,0.0,0.0,0.0



SVM
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976361,0.0,0.0,0.0
Validation,0.976362,0.0,0.0,0.0


In [88]:
reduced_comorb = reduce_training_set(comorb, y)
run_models_cross_val(reduced_comorb, y[reduced_comorb.index], models)

MultinomialNB
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.505328,0.046734,0.598647,0.084677
Validation,0.496664,0.017183,0.500000,0.032933



Random Forest (100 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.555465,0.5,0.277727,0.357101
Validation,0.000820,0.0,0.000000,0.000000



Random Forest (500 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.562011,0.493822,0.604005,0.382781
Validation,0.036440,0.011051,0.500000,0.021582



XGBoost Classifier
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.562011,0.491858,0.616249,0.379743
Validation,0.034399,0.008597,0.500000,0.016862



SVM
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.562011,0.494190,0.618618,0.379924
Validation,0.031126,0.007372,0.500000,0.014500


In [89]:
comorb_ef = pd.concat([comorb, e_factors], axis=1)
run_models_cross_val(comorb_ef, y, models, hr_imputation=True)

MultinomialNB
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.604175,0.459017,0.029377,0.053355
Validation,0.591113,0.405234,0.023402,0.042329



Random Forest (100 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976361,0.0,0.0,0.0
Validation,0.976362,0.0,0.0,0.0



Random Forest (500 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976462,0.004278,1.0,0.008518
Validation,0.976323,0.000000,0.0,0.000000



XGBoost Classifier
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976398,0.001618,0.975,0.003229
Validation,0.976343,0.000000,0.000,0.000000



SVM
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.976361,0.0,0.0,0.0
Validation,0.976362,0.0,0.0,0.0


In [90]:
reduced_comorb_ef = reduce_training_set(comorb_ef, y)
run_models_cross_val(reduced_comorb_ef, y[reduced_comorb_ef.index], models, hr_imputation=True)

MultinomialNB
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.535922,0.206223,0.605572,0.306790
Validation,0.529980,0.098948,0.497143,0.162878



Random Forest (100 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.573104,0.530367,0.577714,0.460789
Validation,0.121127,0.055642,0.496296,0.098874



Random Forest (500 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.596972,0.578577,0.604428,0.535180
Validation,0.212322,0.107932,0.497222,0.171698



XGBoost Classifier
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.583242,0.543734,0.585804,0.495696
Validation,0.176335,0.087509,0.496970,0.147002



SVM
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.578923,0.536509,0.580003,0.485126
Validation,0.169806,0.080983,0.497059,0.138859


Next step to either add more features or... implement a cross validation method that makes up for the difference between positives and negatives

In [91]:
run_models_cross_val(comorb, y[comorb.index], models, ratio_negatives_to_positives = 1)

MultinomialNB
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.494318,0.073783,0.580856,0.128029
Validation,0.494269,0.076243,0.524688,0.129850



Random Forest (100 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.521093,0.931447,0.518037,0.665582
Validation,0.510642,0.922526,0.513821,0.659505



Random Forest (500 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.526093,0.924637,0.521037,0.666349
Validation,0.513919,0.911565,0.513452,0.656759



XGBoost Classifier
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.525866,0.919217,0.520721,0.664591
Validation,0.509826,0.904071,0.511002,0.652391



SVM
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.526593,0.907527,0.521790,0.662384
Validation,0.520060,0.894725,0.518746,0.656237


In [111]:
run_models_cross_val(comorb_ef, y[comorb_ef.index], models, hr_imputation=True, ratio_negatives_to_positives = 1)

MultinomialNB
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.528683,0.384701,0.580982,0.404763
Validation,0.514316,0.364558,0.549884,0.376796



Random Forest (100 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.555641,0.815746,0.544156,0.651681
Validation,0.538076,0.798690,0.533433,0.638589



Random Forest (500 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.576598,0.709121,0.570534,0.630040
Validation,0.539756,0.660445,0.543037,0.592681



XGBoost Classifier
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.557869,0.863251,0.543075,0.666525
Validation,0.542993,0.853280,0.532993,0.655820



SVM
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.555005,0.740311,0.551288,0.623573
Validation,0.540539,0.718872,0.537785,0.606204


In [103]:
weighted_trees = {
    "Random Forest (100 estimators)" : RandomForestClassifier(min_samples_leaf=30, class_weight={"Positive" : 1, "Negative" : .5}),
    "Random Forest (500 estimators)" : RandomForestClassifier(n_estimators = 500, class_weight={"Positive" : 1, "Negative" : .5})
}

In [109]:
run_models_cross_val(comorb_ef, y[comorb_ef.index], weighted_trees, hr_imputation=True, ratio_negatives_to_positives=1)

Random Forest (100 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.511728,1.0,0.511728,0.677009
Validation,0.511874,1.0,0.511874,0.677120



Random Forest (500 estimators)
╠■■■■■■■■■╣


,Accuracy,Recall,Precision,F1
Training,0.546050,0.980247,0.530437,0.688343
Validation,0.511875,0.948761,0.511878,0.664914


We definetley need another set, even though using the cross validation for data imbalancing helped a lot

### Vitals

In [112]:
vitals.info()
vitals.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51695 entries, 0 to 93994
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperature  46961 non-null  float64
 1   pulse        47666 non-null  float64
 2   sys          45990 non-null  float64
 3   dia          45990 non-null  float64
 4   rr           40972 non-null  float64
 5   sats         46959 non-null  float64
dtypes: float64(6)
memory usage: 4.8 MB


(51695, 6)

$ PAM = DIA + \frac{SIS-DIA}{3}$<br>
where PAM = Presión arterial media
SIS = Sístole
DIA = Diástole

As we can see, there are not that many null variables to impute in this set. The worse are